# DSL API ExampleThis notebook demonstrates the **advanced** DSL workflow builder in the Pulse client,including named data sources and explicit wiring of process inputs.

In [ ]:
!pip install pulse-sdk

In [ ]:
from pulse.dsl import Workflow
from pulse.core.client import CoreClient

# Sample data
comments = ["I love this product!", "The experience was okay.", "Worst service ever."]
existing_themes = ["love", "experience", "service"]

In [ ]:
# Build a DSL workflow with sources and explicit wiring
wf = (
    Workflow()
    .source("comments", comments)
    .source("themes", existing_themes)
    .theme_generation(min_themes=3, max_themes=5, fast=True, source="comments")
    .theme_allocation(inputs="comments", themes_from="theme_generation")
    .theme_extraction(inputs="comments", themes_from="theme_generation")
    .sentiment(source="comments", fast=True)
    .sentiment(source="theme_extraction", fast=True)
    .cluster(source="comments", fast=True)
)

In [ ]:
# Visualize the workflow DAG
wf.graph()

In [ ]:
# Execute the DSL workflow
client = CoreClient()  # configure credentials if needed
results = wf.run(client=client)

In [ ]:
# Inspect theme generation results
df_gen = results.theme_generation.to_dataframe()
df_gen

In [ ]:
# Theme allocation assignments
alloc = results.theme_allocation
alloc_df = alloc.assign_single()
alloc_df

In [ ]:
# Theme extraction outputs
df_extr = results.theme_extraction.to_dataframe()
df_extr

In [ ]:
# Sentiment on original comments
results.sentiment.summary()

In [ ]:
# Cluster labels on comments
results.cluster.kmeans(n_clusters=2)